In [46]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [47]:
IMAGE_SIZE = 28
NUM_CHANNELS =1
CONV1_SIZE = 5
CONV1_KERNAL_NUM = 32
CONV2_SIZE =5
CONV2__KERNAL_NUM = 64
FC_SIZE = 512
OUTPUT_NODE = 10

# Forward Propogation

In [48]:
def get_weight(shape,regularizer):
    w = tf.Variable(tf.truncated_normal(shape,stddev = 0.1))
    if regularizer != None:
        tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b

In [49]:
def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides = [1,1,1,1], padding= 'SAME')


In [50]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1],padding = 'SAME')

In [51]:
def forward(x, train,regularizer):
    conv1_w = get_weight([CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_KERNAL_NUM],regularizer)
    conv1_b = get_bias(CONV1_KERNAL_NUM)
    conv1 = conv2d(x,conv1_w)
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1,conv1_b))
    pool1 = max_pool_2x2(relu1)
    
    conv2_w = get_weight([CONV2_SIZE,CONV2_SIZE,CONV1_KERNAL_NUM,CONV2_KERNAL_NUM],regularizer)
    conv2_b = get_bias(CONV2_KERNAL_NUM)
    conv2 = conv2d(pool1,conv2_w)
    relu2 = tf.nn.relu(conv2+conv2_b)
    pool2 = max_pool_2x2(relu2)
    # change the 3d tensor to 2d tensor, and feed the data to the FC layer.
    pool_shape = pool2.get_shape().as_list()
    nodes = pool_shape[1]*pool_shape[2]*pool_shape[3]
    reshaped = tf.reshape(pool2,[pool_shape[0],nodes])
    
    fc1_w = get_weight([nodes,FC_SIZE],regularizer)
    fc1_b = get_bias([FC_SIZE])
    fc1 = tf.nn.relu(tf.matmul(reshaped,fc1_w)+fc1_b)
    if train: 
        fc1 = tf.nn.dropout(fc1,0.5)
        
    fc2_w = get_weight([FC_SIZE,OUTPUT_NODE],regularizer)
    fc2_b = get_bias([OUTPUT_NODE])
    y = tf.matmul(fc1,fc2_w)+fc2_b
    return y

# Backpropogation

In [52]:
BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.005
LEARNING_RATE_DECAY = 0.99
REGULARIZER = 0.0001
STEPS = 50000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = "./lenet5/model"
MODEL_NAME = 'mnist_model'

In [53]:
mnist = input_data.read_data_sets('./data/',one_hot = True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [58]:
def backward(mnist):
    x = tf.placeholder(tf.float32,shape = [BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS])
    y = tf.placeholder(tf.float32,shape = [BATCH_SIZE,OUTPUT_NODE])
    y_ = forward(x,True,REGULARIZER)
    
    global_step = tf.Variable(0,trainable =False)
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y_, labels = tf.argmax(y,1))
    cem =tf.reduce_mean(ce)
    loss = cem + tf.add_n(tf.get_collection('losses'))
    
    learning_rate = tf.train.exponential_decay(learning_rate = LEARNING_RATE_BASE,global_step = global_step,decay_steps = mnist.train.num_examples/BATCH_SIZE,decay_rate = LEARNING_RATE_DECAY,staircase = True)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
    
    ema = tf.train.ExponentialMovingAverage(decay = MOVING_AVERAGE_DECAY,num_updates = global_step)
    ema_op = ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_step,ema_op]):
        train_op = tf.no_op(name='train')
    
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        
        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess,ckpt.model_checkpoint_path)
        
        for i in range(STEPS):
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            reshaped_xs = np.reshape(xs,[BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS])
            _,loss_value,step = sess.run([train_op,loss,global_step],feed_dict={x:reshaped_xs,y:ys})
            
            
            if i%100 ==0:
                print('After %d training steps, loss on training batch is %g'%(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step = global_step)
            

In [ ]:
tf.reset_default_graph()
backward(mnist)

INFO:tensorflow:Restoring parameters from ./lenet5/model/mnist_model-0
After 1 training steps, loss on training batch is 1.05272
After 101 training steps, loss on training batch is 1.027
After 201 training steps, loss on training batch is 1.06863
After 301 training steps, loss on training batch is 1.00309


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/anaconda3/lib/python3.6/asyncio/selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ValueError: embedded null byte


After 401 training steps, loss on training batch is 0.993359
After 501 training steps, loss on training batch is 0.840061
Instructions for updating:
Use standard file APIs to delete files with this prefix.
After 601 training steps, loss on training batch is 1.0009
After 701 training steps, loss on training batch is 0.877063
After 801 training steps, loss on training batch is 0.962802
After 901 training steps, loss on training batch is 0.863011
After 1001 training steps, loss on training batch is 0.908407
After 1101 training steps, loss on training batch is 0.866983
After 1201 training steps, loss on training batch is 0.816147
After 1301 training steps, loss on training batch is 1.00875
After 1401 training steps, loss on training batch is 0.926187
After 1501 training steps, loss on training batch is 0.807646
After 1601 training steps, loss on training batch is 0.843769
After 1701 training steps, loss on training batch is 0.802828


Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/asyncio/events.py", line 145, in _run
    self._callback(*self._args)
  File "/anaconda3/lib/python3.6/asyncio/selector_events.py", line 130, in _read_from_self
    data = self._ssock.recv(4096)
ValueError: embedded null byte


After 1801 training steps, loss on training batch is 0.894504
After 1901 training steps, loss on training batch is 0.908394
After 2001 training steps, loss on training batch is 0.791468
After 2101 training steps, loss on training batch is 0.807414
After 2201 training steps, loss on training batch is 0.800616
After 2301 training steps, loss on training batch is 0.90492
After 2401 training steps, loss on training batch is 0.781209
